In [1]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 34.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [6]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [7]:
df.shape

(40000, 2)

In [8]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [9]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [10]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [11]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [12]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [13]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 1.98 s, sys: 32.1 ms, total: 2.01 s
Wall time: 2.06 s


In [14]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [16]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 5.02 s, sys: 39.8 ms, total: 5.06 s
Wall time: 2.72 s


In [17]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [18]:
coffee_reviews_df.shape

(4981, 2)

In [19]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [20]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [21]:
!ollama pull gemma3:270m

In [22]:
!ollama list

NAME           ID              SIZE      MODIFIED     
gemma3:270m    e7d36fb2c3b3    291 MB    1 second ago    


In [23]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ ยินดีที่ได้รู้จักค่ะ 😊




In [24]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T16:33:40.360148102Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [25]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [26]:
llm = OllamaLLM(model="gemma3:270m")

In [27]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ ยินดีที่ได้รู้จักนะคะ 😊



In [28]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [29]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [30]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [31]:
chain = prompt | llm

In [32]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [33]:
response

'```python\ndef get_sum_of_numbers(numbers):\n  """\n  หาผลรวมของตัวเลข 1 ถึง 10\n  """\n  sum = 0\n  for number in numbers:\n    sum += number\n  return sum\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = get_sum_of_numbers(numbers)\nprint(result)\n```\n\n**คำอธิบาย:**\n\n1. **`def get_sum_of_numbers(numbers):`**:  ชื่อของฟังก์ชันนี้คือ `get_sum_of_numbers` ซึ่งเป็นฟังก์ชันที่ใช้ในการหาผลรวมของตัวเลขในคอลัมน์ `numbers`\n2. **`sum = 0`**:  ตัวแปร `sum` จะถูกวางในถ้านitrile (ถ้านitrile คือถ้านitrile ที่ถูกใช้ในการคำนวณผลรวม)  ตัวแปรนี้จะถูกนำไปใส่ในถ้านitrile\n3. **`for number in numbers:`**:  นี่คือ loop ที่จะวนซ้ำผ่านทุกตัวเลขในคอลัมน์ `numbers`\n4. **`sum += number`**:  ในแต่ละรอบของ loop จะทำการเพิ่มค่าของตัวแปร `sum` ลงในถ้านitrile  `sum` จะถูกนำไปใส่ในถ้านitrile\n5. **`return sum`**:  หลังจากที่ loop finished,  `sum` จะเป็นค่าที่ได้จาก `get_sum_of_numbers` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่\n6. **`nu

In [34]:
from IPython.display import display, Markdown

In [35]:
display(Markdown(response))

```python
def get_sum_of_numbers(numbers):
  """
  หาผลรวมของตัวเลข 1 ถึง 10
  """
  sum = 0
  for number in numbers:
    sum += number
  return sum

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = get_sum_of_numbers(numbers)
print(result)
```

**คำอธิบาย:**

1. **`def get_sum_of_numbers(numbers):`**:  ชื่อของฟังก์ชันนี้คือ `get_sum_of_numbers` ซึ่งเป็นฟังก์ชันที่ใช้ในการหาผลรวมของตัวเลขในคอลัมน์ `numbers`
2. **`sum = 0`**:  ตัวแปร `sum` จะถูกวางในถ้านitrile (ถ้านitrile คือถ้านitrile ที่ถูกใช้ในการคำนวณผลรวม)  ตัวแปรนี้จะถูกนำไปใส่ในถ้านitrile
3. **`for number in numbers:`**:  นี่คือ loop ที่จะวนซ้ำผ่านทุกตัวเลขในคอลัมน์ `numbers`
4. **`sum += number`**:  ในแต่ละรอบของ loop จะทำการเพิ่มค่าของตัวแปร `sum` ลงในถ้านitrile  `sum` จะถูกนำไปใส่ในถ้านitrile
5. **`return sum`**:  หลังจากที่ loop finished,  `sum` จะเป็นค่าที่ได้จาก `get_sum_of_numbers` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่
6. **`numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`**:  นี่คือตัวอย่างการใช้งานฟังก์ชันนี้  `numbers` คือคอลัมน์ที่แสดงถึงตัวเลขในคอลัมน์ `numbers`  `numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]`  นี่คือตัวอย่างตัวเลขที่ต้องการหาผลรวมของ

**การใช้งาน:**

เมื่อคุณเรียก `get_sum_of_numbers(numbers)` คุณจะได้รับผลลัพธ์ที่แสดงผลรวมของตัวเลขในคอลัมน์ `numbers`  ซึ่งเป็นค่าที่ได้จากการคำนวณ  ซึ่งจะเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  ถ้าคุณต้องการให้ผลลัพธ์นี้ถูกนำกลับมาใส่ในถ้านitrile  คุณสามารถทำได้โดยการใช้ `numbers.sum()`  ซึ่งจะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile  แล้วทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `sum` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  `numbers.sum()`  จะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile และทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `sum` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  `numbers.sum()`  จะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile และทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `sum` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  `sum()`  จะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile และทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `sum` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  `numbers.sum()`  จะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile และทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `sum()` จะถูกนำกลับมาใส่ในถ้านitrile  `sum` จะถูกคืนค่าเป็นตัวแปรที่ใช้ในการหาผลรวมของตัวเลขใหม่  `sum()`  จะทำการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile และทำการนำค่าที่ได้จาก `sum()` ไปใส่ในถ้านitrile  `ถ้านitrile` คือถ้านitrile ที่ถูกใช้ในการคำนวณผลรวมของตัวเลข

**ข้อดีของฟังก์ชัน `get_sum_of_numbers()`:**

* **ข้อดี:**  ฟังก์ชันนี้มีข้อดีคือการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile ทำให้การคำนวณผลรวมของตัวเลขใหม่ง่ายขึ้น
* **ข้อดี:**  ฟังก์ชันนี้มีข้อดีคือการแปลงคอลัมน์ `numbers` ให้เป็นถ้านitrile ทำให้การคำนวณผลรวมของตัวเลขใหม่ง่ายขึ้น

**ข้อควรระวัง:**

*  ฟังก์ชัน `get_sum_of_numbers()`  อาจมีข้อจำกัดในบางกรณี เช่น การคำนวณผลรวมของตัวเลขที่ซับซ้อน หรือการคำนวณผลรวมของตัวเลขที่ผิดพลาด
*  การใช้ `sum()`  ในการคำนวณผลรวมของตัวเลขใหม่อาจทำให้เกิดข้อผิดพลาดได้หากตัวเลขที่อยู่ในคอลัมน์ `numbers` มีค่าที่แตกต่างกัน

หวังว่าคำอธิบายนี้จะเป็นประโยชน์!


In [36]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [37]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [38]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [39]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [40]:
parser_chain = StrOutputParser()

In [41]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [42]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [43]:
result

'ถูกต้องแล้วครับ\n'

In [44]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [45]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [46]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [47]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [48]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [49]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [50]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [51]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [52]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 35.6 ms, sys: 2.95 ms, total: 38.5 ms
Wall time: 5.07 s


In [53]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหารใหญ่", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มันคาว", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "เกินไป", "รับไม่ไหว", "เลิกกิน", "ขนมหวานให้กินฟรี", "ขนมไทย", "ขนมข้าว"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "รสชาติ",
    "กลิ่น",
    "เมพพพ",
    "ซากอารยธรรมลาเต้",
    "ซอยไรจำไม่ได้",
    "แต่อยู่ตรงข้าม Villa"
  ]
}
```



In [54]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [55]:
!ollama list

NAME                                  ID              SIZE      MODIFIED      
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    2 seconds ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    2 minutes ago    


In [56]:
llm3 = OllamaLLM(model="scb10x/typhoon2.1-gemma3-4b")

In [57]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [58]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 112 ms, sys: 4.3 ms, total: 117 ms
Wall time: 17.1 s


In [59]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณบอกมาเพื่อดึงคำหลักจากรีวิวนี้:

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงร้านอาหารขนาดใหญ่ที่มีข้อดีและข้อเสียหลายอย่าง โดยเน้นที่ราคาที่สูงเกินไปเมื่อเทียบกับคุณภาพอาหารบางเมนู แต่ก็มีเมนูที่อร่อยและน่าจะกลับมาอีก

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้านอาหาร
*   ราคาอาหาร
*   รสชาติอาหาร (โดยเฉพาะเมนูลาบไข่ต้ม, ทอดมันหัวปลี, พะแนงห่อไข่)
*   ขนมหวานฟรี

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

จากหัวข้อหลักที่กล่าวถึง ผมเลือกคำหลักดังนี้:

1.  ราคา
2.  อาหาร (โดยเฉพาะเมนูทอดมันหัวปลีและพะแนงห่อไข่)
3.  ขนาดร้าน

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคา", "อาหาร", "ขนาดร้าน"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากมีอะไรให้ผมช่วยเพิ่มเติม บอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text แน่นอนค

In [60]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [61]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [62]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [63]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 141 ms, sys: 7.04 ms, total: 148 ms
Wall time: 12.3 s


In [64]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'รสชาติ']

In [65]:
results[1]['text']['keywords']

['กาแฟ', 'บรรยากาศ', 'บริการ']

In [66]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [67]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 144 ms, sys: 4.09 ms, total: 148 ms
Wall time: 13.1 s


In [68]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'รสชาติ'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [69]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [70]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [71]:
inputs = [{"review": r} for r in sample_reviews]

In [72]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 20.2 ms, sys: 2.9 ms, total: 23.1 ms
Wall time: 3.29 s


In [73]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [77]:
from tqdm import tqdm

sample_reviews_100 = coffee_reviews_df['column0'].iloc[0:200].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]

BATCH_SIZE = 8

results_100 = []

In [78]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:09<00:00, 10.00s/it]


In [79]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part1.csv", index=False, encoding='utf-8-sig')

In [80]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[200:400].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [81]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [04:01<00:00,  9.64s/it]


In [82]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part2.csv", index=False, encoding='utf-8-sig')

In [83]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[400:600].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [84]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [03:56<00:00,  9.44s/it]


In [85]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part3.csv", index=False, encoding='utf-8-sig')

In [86]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[600:800].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [87]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [03:56<00:00,  9.46s/it]


In [88]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part4.csv", index=False, encoding='utf-8-sig')

In [89]:
sample_reviews_100 = coffee_reviews_df['column0'].iloc[800:1000].tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]
BATCH_SIZE = 8
results_100 = []

In [90]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})


Extracting keywords: 100%|██████████| 25/25 [03:49<00:00,  9.18s/it]


In [91]:
import pandas as pd
df = pd.DataFrame(results_100)
df.to_csv("results_part5.csv", index=False, encoding='utf-8-sig')

In [92]:
from tqdm import tqdm

In [93]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 25/25 [03:47<00:00,  9.09s/it]


In [94]:
for result in results_100[0:10]:
    print(result['text']['keywords'])

ช็อกโกแลต, เค้ก, ขนมหวาน
สนามบิน, ร้านอาหาร, บรรยากาศ
กาแฟ, เค้ก, บรรยากาศ
อาหารอิตาเลียน, บรรยากาศ, พิซซ่า
บรรยากาศ, ฟิวชั่น, ราคา
บรรยากาศ, ส้มตำ, รสชาติ
กาแฟ, เอ็กซ์เพรสโซ่, กลิ่น
วาฟเฟิล, กาแฟ, ไซรัป
กาแฟ, โปรโมชั่น, คนเยอะ
อาหารฝรั่ง, ชีสทอด, ฮันนี่โทส


In [95]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [96]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [97]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [98]:
context_extract = context_prompt | llm3 | context_parser

In [99]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [100]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [101]:
results_100[0]['text']['keywords'].split(',')

['ช็อกโกแลต', ' เค้ก', ' ขนมหวาน']

In [102]:
context_inputs = []

for review_data in tqdm(results_100, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 400/400 [00:00<00:00, 190325.76it/s]


In [103]:
len(context_inputs)

1200

In [104]:
context_results = []
for i in tqdm(range(len(results_100)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_100[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 400/400 [27:29<00:00,  4.12s/it]


In [105]:
results_100[:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา']}]

In [106]:
sentiment_inputs = []

for review_data in tqdm(results_100, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 400/400 [00:00<00:00, 255711.26it/s]


In [107]:
len(sentiment_inputs)

1200

In [108]:
sentiment_inputs[:6]

[{'message': 'ช็อกโกแลต'},
 {'message': 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ'},
 {'message': 'ขนมอร่อย'},
 {'message': 'สนามบิน'},
 {'message': 'ร้านอาหารเล็กๆ'},
 {'message': 'บรรยากาศดีเหมือนอยู่ในท้องนา'}]

In [109]:
for i in tqdm(range(len(results_100)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_100[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 400/400 [17:31<00:00,  2.63s/it]


In [110]:
results_100[0:2]

[{'review': 'Chocolate crape cake with strawberry 145฿\nDark chocolate Ganache cake 145฿\n\nบริการแบบ งงๆ แบบมีหลายเคาร์เตอร์ \n\nขนมอร่อย ร้านตกแต่งสวย มีทั้งอาหาร ขนม กาแฟ มีของ grocery ให้ซื้อไป cook เองที่บ้านได้ \n\nอิ่มคาวมาจากร้านข้างบนแล้ว มาต่อแค่ขนมหวาน 2 ชิ้นเท่านั้น แต่ฟินง่ะ\n\nแอบอ่านรีวิวก่อนหน้า ส่วนใหญ่จะ complain อาหารคาว',
  'text': {'keywords': 'ช็อกโกแลต, เค้ก, ขนมหวาน'},
  'contexts': ['ช็อกโกแลต', 'เค้กช็อกโกแลตครีมเค้กช็อกโกแลตดำ', 'ขนมอร่อย'],
  'sentiment': ['neutral', 'neutral', 'positive']},
 {'review': 'ถ้ามีโอกาสมาสนามบิน ห้ามพลาด ร้านอาหารเล็กๆ มีให้เลือกทั้งก๋วยเตี๋ยว ส้มตำ น้ำตก อาหารจานเดียว หรือกาแฟสด น้ำผลไม้ต่างๆ ให้เลือกหลากหลาย อีกทั้งบรรยากาศดีเหมือนอยู่ในท้องนา ภายในสนามบินยังมีสินค้าจำหน่ายย่อมๆ ให้ซื้อเป็นที่ระลึก และยังมีสวนสัตว์ย่อมๆ ไว้ให้เพลิดเพลินด้วย',
  'text': {'keywords': 'สนามบิน, ร้านอาหาร, บรรยากาศ'},
  'contexts': ['สนามบิน', 'ร้านอาหารเล็กๆ', 'บรรยากาศดีเหมือนอยู่ในท้องนา'],
  'sentiment': ['neutral', 'neutral', 'positive']}]

In [111]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_100
])

In [112]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')